In [1]:
import urllib.request
import json
import dml
import prov.model
import datetime
import uuid
import statistics
import pandas as pd
from bson.code import Code

'''Retrieve some datasets'''

startTime = datetime.datetime.now()

# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate('anuragp1_jl101995', 'anuragp1_jl101995')

# When Trial is True, perform function on random sample of size SIZE)
SIZE = 100

def get_collection(coll_name):
    # Collection for station sample
    sample_coll_name = coll_name.split(
        'repo.anuragp1_jl101995.')[1] + '_sample'
    repo.dropPermanent(sample_coll_name)
    repo.createPermanent(sample_coll_name)


    return eval(coll_name)

print('Loading citibike_data from Mongo')
citibike_data = repo.anuragp1_jl101995.citibike.find()

print('Cleaning citibike_data')
cb_data = []
for entry in citibike_data:
    date = entry['starttime']
    date = date[:date.index(' ')]
    if '-' in date:
        date = date.split('-')
    elif '/' in date:
        date = date.split('/') 
    # 5/31/2015 12:12:12
    # date = ['5', '31, '2015']
    # [2015 - 4 - 10]

    if len(date[0]) < 2:
        date[0] = '0' + date[0]
    if len(date[1]) < 2:
        date[1] = '0' + date[1]
    if len(date[2]) < 2:
        date[2] = '0' + date[2]

    if date[0] in ['2014', '2015', '2016']:
        new_date = date[0] + date[1] + date[2]
        cb_data.append([new_date, entry['start station name']])

    elif date[2] in ['2014', '2015', '2016']:
        new_date = date[2] + date[0] + date[1]
        cb_data.append([new_date, entry['start station name']])

print('Creating pandas dataframe from cleaned citibike_data')
cb_df = pd.DataFrame(cb_data, columns = ['date', 'start_station_name'])


print('Getting citibike totals for each day')
cb_df['Count'] = cb_df['start_station_name']
counted_cb_df = pd.DataFrame(cb_df.groupby(['date','start_station_name'], as_index=False)['Count'].count())
counted_cb_df

Loading citibike_data from Mongo
Cleaning citibike_data
Creating pandas dataframe from cleaned citibike_data
Getting citibike totals for each day


,date,start_station_name,Count
0,20140101,1 Ave & E 15 St,60
1,20140101,1 Ave & E 18 St,32
2,20140101,1 Ave & E 30 St,18
3,20140101,1 Ave & E 44 St,12
4,20140101,10 Ave & W 28 St,18
5,20140101,11 Ave & W 27 St,10
6,20140101,11 Ave & W 41 St,25
7,20140101,12 Ave & W 40 St,9
8,20140101,2 Ave & E 31 St,62
9,20140101,2 Ave & E 58 St,17


In [2]:
print('Starting')
repo.dropPermanent('citibike_startstation_byday')
repo.createPermanent('citibike_startstation_byday')

for index, row in counted_cb_df.iterrows():
    #datestring = str(row[0])[4:6] + '/' + str(row[0])[6:8] + '/' + str(row[0])[0:4]
    insert_this = {'Date': row[0], 'Start_Station' : row[1], 'Count': int(row[2])}
    repo.anuragp1_jl101995.citibike_startstation_byday.insert_one(insert_this)
    
print('Finished')

Starting
Finished
